# Assignment: Preparing Data for Analysis (Modified Titanic)

![](https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/%20files/hw.png)

In [272]:
#Import the libraries
import numpy as np
import pandas as pd

In [273]:
df = pd.read_csv('https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/hw/titanic_training_dataset_v2.csv')

In [274]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     865 non-null    float64
 2   Pclass       828 non-null    float64
 3   Name         866 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        858 non-null    float64
 7   Parch        891 non-null    int64  
 8   Ticket       852 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     803 non-null    object 
dtypes: float64(5), int64(2), object(5)
memory usage: 83.7+ KB


In [275]:
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0,PC 17599,71.2833,C85,C


# 1) Load data & review the data

<font color='blue'>Q1) What is the shape of this dataset? (rows & columns)</font>

In [276]:
df.shape

(891, 12)

# 2) Drop unqualified variables

*   Drop variables with missing > 50%
*   Drop categorical variables with flat values > 70% (variables with the same value in the same column)

<font color='blue'>Q2) How many columns do we have left?</font>




In [277]:
half_count = len(df) / 2
df = df.dropna(thresh=half_count,axis=1) # Drop any column with more than 50% missing values
df.shape

(891, 11)

In [278]:
for col in df.columns:
    print(df[col].value_counts(normalize=True,dropna=False).iloc[0],col)
df.drop(columns=['Parch'],inplace=True)
df.shape

0.001122334455667789 PassengerId
0.5982042648709316 Survived
0.5095398428731762 Pclass
0.028058361391694726 Name
0.6475869809203143 Sex
0.19865319865319866 Age
0.6599326599326599 SibSp
0.7609427609427609 Parch
0.04377104377104377 Ticket
0.04826038159371493 Fare
0.6509539842873177 Embarked


(891, 10)

# 3) Remove all rows with missing target (the variable "Survived")

<font color='blue'>Q3) How many rows do we have left?</font>

In [279]:
df.dropna(subset=['Survived'],inplace=True)
df.shape

(865, 10)

# 4) Handle outliers 

For the variable “Fare”, replace outlier values with the boundary values


*   If value < (Q1 - 1.5IQR), relace with (Q1 - 1.5IQR)
*   If value > (Q3 + 1.5IQR), relace with (Q3 + 1.5IQR)

<font color='blue'>Q4) What is the mean of “Fare” after replacing the outliers (round 2 decimal points)?</font>



In [280]:
q75, q25 = np.percentile(df.Fare.dropna(), [75 ,25])
iqr = q75 - q25

min = q25 - (iqr*1.5)
max = q75 + (iqr*1.5)
df.loc[df["Fare"] < min, "Fare"] = min
df.loc[df["Fare"] > max, "Fare"] = max

print((df.Fare > max).sum())
print((df.Fare < min).sum())
print(df.Fare.mean())
df

0
0
24.038996878612718


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Ticket,Fare,Embarked
0,1,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,A/5 21171,7.2500,S
1,2,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,PC 17599,65.6563,C
2,3,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,STON/O2. 3101282,7.9250,S
3,4,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,113803,53.1000,S
4,5,0.0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...
886,887,0.0,2.0,"Montvila, Rev. Juozas",male,27.0,0.0,211536,13.0000,S
887,888,1.0,1.0,"Graham, Miss. Margaret Edith",female,19.0,0.0,112053,30.0000,S
888,889,0.0,3.0,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1.0,W./C. 6607,23.4500,S
889,890,1.0,1.0,"Behr, Mr. Karl Howell",male,26.0,0.0,111369,30.0000,C


# 5) Impute missing value



*   Show the number of missing values in each variable
*   Impute missing values with mean & mode
*   Show the number of missing values again after missing value imputation

<font color='blue'>Q5) Which variable has the largest number of missing values?</font>



In [281]:
print(df.isnull().sum())
# after mean
df['Age'].fillna((df['Age'].mean()), inplace=True)
# after mode
df.fillna(df.mode().iloc[0], inplace=True)
print(df.isnull().sum())

PassengerId      0
Survived         0
Pclass          63
Name            25
Sex              0
Age            172
SibSp           33
Ticket          39
Fare             0
Embarked        87
dtype: int64
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


# 6) Convert categorical to numeric values



*   Drop the variables “Name” & “Ticket”
*   For the variables “Sex” & “Embarked”, perform the dummy coding and drop the first level. Also, drop those original variables (“Sex” & “Embarked”)

<font color='blue'>Q6) How many columns do we have?</font>



In [282]:
df=df.drop(['Name','Ticket'], axis=1)
df.shape

(865, 8)

In [283]:
nominal_columns = ["Embarked", "Sex"]
dummy_df = pd.get_dummies(df[nominal_columns], drop_first=True)
df = pd.concat([df,dummy_df], axis=1)
df.drop(nominal_columns, axis=1,inplace=True)
df.shape

(865, 9)

# 7) Partition data



*   Split train/test split with stratification using 70%:30% and random seed with 12345
*   Show a proportion between survived (1) and died (0) in all data sets (total data, train, test)

<font color='blue'>Q7) What is a proportion between survived (1) and died (0) in the training data?</font>



In [284]:
from sklearn.model_selection import train_test_split
df.dropna(subset=['Survived'],inplace=True)
x=df.copy()
y=x.pop('Survived')
x_train,x_test,y_train,y_test = train_test_split(x,y,stratify=y,test_size=0.30, random_state=12345)

In [285]:
print(df.Survived.value_counts()[0]/df.Survived.value_counts()[1])

1.605421686746988


In [286]:
print(y_train.value_counts()[0]/y_train.value_counts()[1])

1.6077586206896552


In [287]:

print(y_test.value_counts()[0]/y_test.value_counts()[1])

1.6


In [288]:
print(y_train.shape)
print(y_test.shape)

(605,)
(260,)


In [289]:
x_train

,PassengerId,Pclass,Age,SibSp,Fare,Embarked_Q,Embarked_S,Sex_male
305,306,1.0,0.920000,1.0,65.6563,False,True,True
879,880,1.0,56.000000,0.0,65.6563,False,False,False
381,382,3.0,1.000000,0.0,15.7417,False,False,False
491,492,3.0,21.000000,0.0,7.2500,False,True,True
1,2,1.0,38.000000,1.0,65.6563,False,False,False
...,...,...,...,...,...,...,...,...
362,363,3.0,45.000000,0.0,14.4542,False,False,False
247,248,2.0,24.000000,0.0,14.5000,False,True,False
331,332,1.0,45.500000,0.0,28.5000,False,True,True
110,111,1.0,47.000000,0.0,52.0000,False,True,True


In [290]:
x_test

,PassengerId,Pclass,Age,SibSp,Fare,Embarked_Q,Embarked_S,Sex_male
807,808,3.0,18.000000,0.0,7.7750,False,True,False
324,325,3.0,29.892266,8.0,65.6563,False,True,True
389,390,2.0,17.000000,0.0,12.0000,False,False,False
329,330,1.0,16.000000,0.0,57.9792,False,False,False
779,780,1.0,43.000000,0.0,65.6563,False,True,False
...,...,...,...,...,...,...,...,...
10,11,3.0,4.000000,1.0,16.7000,False,True,False
502,503,3.0,29.892266,0.0,7.6292,True,False,False
0,1,3.0,22.000000,1.0,7.2500,False,True,True
223,224,3.0,29.892266,0.0,7.8958,False,True,True


In [291]:
y_train

305    1.0
879    1.0
381    1.0
491    0.0
1      1.0
      ... 
362    0.0
247    1.0
331    0.0
110    0.0
45     0.0
Name: Survived, Length: 605, dtype: float64

In [292]:
y_test

807    0.0
324    0.0
389    1.0
329    1.0
779    1.0
      ... 
10     1.0
502    0.0
0      0.0
223    0.0
218    1.0
Name: Survived, Length: 260, dtype: float64